In [24]:
import numpy as np
import pandas as pd
import time

np.random.seed(2)

In [25]:
N_STATES = 6 #寻宝路径的长度为6
ACTIONS = ['left','right'] #可用动作
EPSILON = 0.9 #贪婪度
ALPHA = 0.1 #学习率
GAMMA = 0.9 #奖励递减值
MAX_EPISODES = 13 #最大回合数
FRESH_TIME = 0.3 #移动时间间隔

In [26]:
def build_q_table(n_states,actions):
    table = pd.DataFrame(np.zeros((n_states,len(actions))),
                        columns=actions)
    return table

In [27]:
def choose_action(state,q_table):
    state_actions = q_table.iloc[state,:]
    if (np.random.uniform() > EPSILON) or ((state_actions==0).all()):
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_actions.idxmax()
    return action_name

In [28]:
def get_env_feedback(S,A):
    if A == 'right':
        if S == N_STATES - 2:
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:
        R = 0
        if S == 0:
            S_ = S
        else:
            S_ = S -1
    return S_,R

In [29]:
def update_env(S,episode,step_counter):
    env_list = ['-'] * (N_STATES - 1) + ['T']
    if S == 'terminal':
        interaction = 'Episode %s: total_step = %s' % (episode + 1,step_counter)
        print('\r{}'.format(interaction),end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction),end='')
        time.sleep(FRESH_TIME)

In [30]:
def rl():
    q_table = build_q_table(N_STATES,ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S,episode,step_counter)
        while not is_terminated:
            A = choose_action(S,q_table)
            S_,R = get_env_feedback(S,A)
            q_predict = q_table.loc[S,A]
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_,:].max()
            else:
                q_target = R
                is_terminated = True
                
            q_table.loc[S,A] += ALPHA * (q_target - q_predict)
            S = S_
            update_env(S,episode,step_counter+1)
            step_counter += 1
    return q_table

In [31]:
q_table = rl()
print('\r\nQ-table:\n')
print(q_table)

                                
Q-table:

       left     right
0  0.000000  0.004320
1  0.000000  0.025005
2  0.000030  0.111241
3  0.000000  0.368750
4  0.027621  0.745813
5  0.000000  0.000000
